# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. Use this dataset to answer the questions below.

In [5]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)
df.head()

,№ Summer,01 !,02 !,03 !,Total,№ Winter,01 !.1,02 !.1,03 !.1,Total.1,№ Games,01 !.2,02 !.2,03 !.2,Combined total
Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [6]:
for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [7]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
firstCountry = answer_zero() 
type(firstCountry)
firstCountry

pandas.core.series.Series

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [14]:
import numpy as np

def answer_one():
    maxSG = np.max(df['Gold'])
#     return df[df['Gold'] == maxSG].index.tolist()
    return df.index[df['Gold'] == maxSG]
answer_one()

Index(['United States'], dtype='object')

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [27]:
def answer_two():
    diffG = np.abs(df['Gold'] - df['Gold.1'])
    maxDiffG = np.max(diffG)
    return df[(df['Gold'] - df['Gold.1']) == maxDiffG].index
answer_two()

Index(['United States'], dtype='object')

### Question 3
Which country has the biggest difference between their summer and winter gold medal counts relative to their total gold medal count? Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [19]:
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [32]:
def answer_three():
    dfGold = df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    relDiffG = np.max((dfGold['Gold'] - dfGold['Gold.1'])/dfGold['Gold.2'])
    return dfGold[(dfGold['Gold'] - dfGold['Gold.1'])/dfGold['Gold.2'] == relDiffG].index
answer_three()

Index(['Bulgaria'], dtype='object')

### Question 4
Write a function to update the dataframe to include a new column called "Points" which is a weighted value where each gold medal counts for 3 points, silver medals for 2 points, and bronze mdeals for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [33]:
def answer_four():
    df['Points'] = df['Gold.2'] * 3 + df['Silver.2'] * 2 + df['Bronze.2']
    return df['Points']
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

In [34]:
len(df['Points'])

146

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [2]:
import pandas as pd
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [38]:
census_df.where(census_df['SUMLEV'] == 40).dropna()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40.0,3.0,6.0,1.0,0.0,Alabama,Alabama,4779736.0,4780127.0,4785161.0,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
68,40.0,4.0,9.0,2.0,0.0,Alaska,Alaska,710231.0,710249.0,714021.0,...,-1.173489,-1.946424,-3.915107,-14.438910,-10.407475,0.931274,1.818497,-0.757148,-11.271709,-6.881838
98,40.0,4.0,8.0,4.0,0.0,Arizona,Arizona,6392017.0,6392307.0,6408208.0,...,1.327489,5.245740,3.905473,6.219955,6.776501,3.290378,7.337279,6.123606,8.761352,9.335208
114,40.0,3.0,7.0,5.0,0.0,Arkansas,Arkansas,2915918.0,2915958.0,2922394.0,...,1.365312,-0.432402,-0.442153,-1.060966,-0.407735,2.326251,0.646395,0.682189,0.226168,0.901928
190,40.0,4.0,9.0,6.0,0.0,California,California,37253956.0,37254503.0,37334079.0,...,-1.148464,-1.163788,-1.339869,-0.862856,-1.981572,2.761704,2.647127,2.728645,3.743342,2.656065
249,40.0,4.0,8.0,8.0,0.0,Colorado,Colorado,5029196.0,5029324.0,5048254.0,...,5.232828,5.513416,6.903846,7.375183,10.073656,6.968908,7.698805,9.053163,9.764255,12.537918
314,40.0,1.0,1.0,9.0,0.0,Connecticut,Connecticut,3574097.0,3574118.0,3579717.0,...,-3.724400,-5.311208,-4.730271,-7.567093,-7.687268,0.775510,-1.066084,-0.278693,-2.376831,-2.463243
323,40.0,3.0,5.0,10.0,0.0,Delaware,Delaware,897934.0,897936.0,899791.0,...,2.867721,3.594491,3.169689,5.106051,4.490138,5.332723,6.360496,5.939911,8.245757,7.628452
327,40.0,3.0,5.0,11.0,0.0,District of Columbia,District of Columbia,601723.0,601767.0,605126.0,...,11.560071,10.052444,9.457678,1.480094,5.601833,17.028422,15.972111,15.634120,8.378037,12.434838
329,40.0,3.0,5.0,12.0,0.0,Florida,Florida,18801310.0,18804623.0,18849890.0,...,5.556307,5.138184,4.847420,6.958576,10.080932,11.364173,10.676030,10.565882,13.478570,16.528676


In [57]:
def answer_five():
    countyNum = {}
    county_df = census_df[census_df['SUMLEV'] == 50]
    for state in county_df['STNAME'].unique():
        countyNum[state] = np.sum(county_df[county_df['STNAME'] == state].dropna()['COUNTY'])
    countyNum = pd.Series(countyNum)
    return countyNum.where(countyNum == np.max(countyNum)).dropna().index
answer_five()

Index(['Texas'], dtype='object')

In [60]:
countyNum = {}
county_df = census_df[census_df['SUMLEV'] == 50]
for group, frame in county_df.groupby('STNAME'):
    countySum = np.sum(frame['COUNTY'])
    countyNum[group] = countySum
countyNum

{'Alabama': 4489,
 'Alaska': 4340,
 'Arizona': 208,
 'Arkansas': 5625,
 'California': 3364,
 'Colorado': 3983,
 'Connecticut': 64,
 'Delaware': 9,
 'District of Columbia': 1,
 'Florida': 4550,
 'Georgia': 25677,
 'Hawaii': 25,
 'Idaho': 1936,
 'Illinois': 10404,
 'Indiana': 8464,
 'Iowa': 9801,
 'Kansas': 11025,
 'Kentucky': 14400,
 'Louisiana': 4096,
 'Maine': 256,
 'Maryland': 1079,
 'Massachusetts': 196,
 'Michigan': 6889,
 'Minnesota': 7569,
 'Mississippi': 6724,
 'Missouri': 13537,
 'Montana': 3136,
 'Nebraska': 8649,
 'Nevada': 774,
 'New Hampshire': 100,
 'New Jersey': 441,
 'New Mexico': 995,
 'New York': 3844,
 'North Carolina': 10000,
 'North Dakota': 2809,
 'Ohio': 7744,
 'Oklahoma': 5929,
 'Oregon': 1296,
 'Pennsylvania': 4489,
 'Rhode Island': 25,
 'South Carolina': 2116,
 'South Dakota': 4485,
 'Tennessee': 9025,
 'Texas': 64516,
 'Utah': 841,
 'Vermont': 196,
 'Virginia': 35111,
 'Washington': 1521,
 'West Virginia': 3025,
 'Wisconsin': 5119,
 'Wyoming': 529}

In [64]:
county_df = census_df[census_df['SUMLEV'] == 50]
countyNum = county_df.set_index('STNAME').groupby(level = 0)['COUNTY'].agg({'COUNTY': np.sum})

C:\Users\xuteng\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [66]:
countyNum[countyNum['COUNTY'] == np.max(countyNum['COUNTY'])].index

Index(['Texas'], dtype='object', name='STNAME')

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)?

*This function should return a list of string values.*

In [67]:
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [3]:
def answer_six():
    counties_df = census_df[census_df['SUMLEV'] == 50]
    top_counties_df = counties_df.sort_values(by=['STNAME','CENSUS2010POP'],ascending=False).groupby('STNAME').head(3)
    ans = top_counties_df.groupby('STNAME').sum().sort_values(by='CENSUS2010POP').head(3).index.tolist()
    return ans
answer_six()

['Wyoming', 'Vermont', 'North Dakota']

### Question 7
Which county has had the largest change in population within the five year period (hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all five columns)?

*This function should return a single string value.*

In [6]:
def answer_seven():
    counties_df = census_df[census_df['SUMLEV'] == 50]
    counties_df['changePop'] = abs(counties_df['POPESTIMATE2011'] - counties_df['POPESTIMATE2010']) + abs(counties_df['POPESTIMATE2012'] - counties_df['POPESTIMATE2011']) + abs(counties_df['POPESTIMATE2013'] - counties_df['POPESTIMATE2012']) + abs(counties_df['POPESTIMATE2014'] - counties_df['POPESTIMATE2013']) + abs(counties_df['POPESTIMATE2015'] - counties_df['POPESTIMATE2014'])
    return counties_df.sort_values(by = 'changePop', ascending = False)


C:\Users\xuteng\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,changePop
2667,50,3,7,48,201,Texas,Harris County,4092459,4093076,4108187,...,3.518819,4.820913,5.243613,3.794291,7.154417,8.915715,10.683269,11.850629,10.303814,429841
209,50,4,9,6,37,California,Los Angeles County,9818605,9818700,9826009,...,-4.071266,-4.332318,-5.394299,-6.027694,-0.007808,0.377510,0.578049,0.215038,-0.437481,344283
106,50,4,8,4,13,Arizona,Maricopa County,3817117,3817357,3825597,...,9.232973,7.879873,8.318210,9.123309,4.726436,11.527593,10.390177,11.203958,11.985998,342350
227,50,4,9,6,73,California,San Diego County,3095313,3095308,3104386,...,0.235513,-0.246567,0.657077,-2.855045,4.382515,5.773072,4.745940,5.921091,2.770965,195135
372,50,3,5,12,86,Florida,Miami-Dade County,2496435,2498017,2508171,...,-6.908149,-8.130040,-10.464276,-12.205475,23.110902,6.891198,6.633403,6.087936,4.154777,184946


In [8]:
a = answer_seven()
a[0]['COUNTY']

C:\Users\xuteng\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: 0

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [106]:
def answer_eight():


,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
